<a href="https://colab.research.google.com/github/wdempsey/AI4Health-Online-Experimentation/blob/main/part3_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 3: Synthetic HeartSteps and personalization


In [ ]:
## Import necessary 
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression

#Part 1: Overview on Contextual Bandits

- For each person in a study, let $t=1,\ldots, T$ denote a sequence of decision points.  
- At each decision time $t$,  we observe a state variable $S_t \in \mathbb{R}^p$.  
- After observing the state variable $S_t$, the _agent_ decides to take action $A_t \in \mathcal{A}$.  
- After observing state $S_t$ and taking action $A_t$, the agent receive a reward $R_t$ given by
$$
R_t = r(S_t, A_t) + \epsilon_t
$$
where $r(c,a)$ is a function that maps the state-action pair onto the real line and $\epsilon_t$ is a random error term, e.g., $\mathbb{E} [\epsilon_t] = 0$. 
- The triple (context, action, reward) at a sequence of decision points defines a _contextual bandit_ setting.  
- Here, the goal is to maximize the expected reward at every time point $\mathbb{E}[R_t \mid S_t, A_t=a] = r(S_t, a)$. 
- If we knew the reward function $r: \mathcal{S} \times \mathcal{S} \to \mathbb{R}$, then the optimal action given state $s$ is
$$
a^\star (s) = \max_{a \in \mathcal{A}} r(s, a)
$$

### Linear Contextual Bandit

- Assume that the reward structure follows
$$
r(s,a) = x(s,a)^\top \beta 
$$
where $x(s,a) \in \mathbb{R}^{p}$ is a $p$-dimensional summary of the state and $\beta \in \mathbb{R}^p$ is an unknown parameter.
- Before, we just wanted to build a good policy after collecting data on some participants.  
- Suppose now, we wish to minimize our __regret__ in making sub-optimal decisions on an individual

### 

First 10 entries of state (2D), action, and reward
[[ 0.4889732   0.          0.          0.0970272 ]
 [-1.65526392  1.          0.         -2.15776888]
 [ 0.88552154  0.          0.          1.29955221]
 [-1.21393632  1.          0.          0.97508805]
 [-1.79251348  1.          0.         -1.62145004]
 [ 1.42910115  0.          1.          1.68040017]
 [-2.05180643  1.          0.         -3.29626183]
 [ 0.2045409   1.          0.          1.70142421]
 [ 2.92629192  1.          0.          1.87006771]]


True coefficients using linear model
[ 1.   0.3  0.5 -0.7]
Fitted coefficients using linear model
[ 1.08134331  0.49970343  0.50450428 -0.64743794]


## Question 1: How would you use the warm-start policy 

## Question 2: What are some potential pitfalls in this approach?


## Exercise:  Code up a simple Thompson sampling algorithm for the next individual in the MRT.

In [ ]:
## Reading in the data


## ADD CODE FOR EXPLORATORY DATA ANALYSIS HERE


## Part 1d: Going beyond Bandits

- Bandit algorithms maximize immediate reward
- This ignores potential longer term impacts of treatment 
- __Question__: What are some 

A __Markov Decision Process__ is 



```
# This is formatted as code
```



# Part 2: Constrained Optimization